In [1]:
import pandas as pd
import configparser
import boto3

In [2]:
# import glob
# files = glob.glob('../../../自己啓発/スマート農業/青果物市況情報/ダウンロードフォルダ_1225_1132/*')

# dfs = []
# for file in files:
#     df_temp = pd.read_csv(file,encoding='shift_jis')
#     dfs.append(df_temp)
# df = pd.concat(dfs)
# df.to_parquet('../../../自己啓発/スマート農業/青果物市況情報/data_all.parquet')

In [3]:
df = pd.read_parquet("../../../自己啓発/スマート農業/青果物市況情報/data_all.parquet")
df.head()

,年,月,日,曜日,市場名,市場コード,品目名,品目コード,産地名,産地コード,品目計（ｔ：トン）,入荷量（ｔ：トン）,高値（円）,中値（円）,安値（円）,等級,階級,品名,重量（kg）,概況
0,2019,9,17,火,盛岡,3300,だいこん,30100,青 森,2.0,37.3,21.7,864,-,756,A,L,青首,10.0,強い
1,2019,9,17,火,盛岡,3300,だいこん,30100,岩 手,3.0,NaN,14.0,864,540,324,A,LL,None,10.0,None
2,2019,9,17,火,盛岡,3300,かぶ,30200,青 森,2.0,1.9,1.9,1296,1080,1080,A,LL,小かぶ,5.0,保合
3,2019,9,17,火,盛岡,3300,にんじん,30300,北海道,1.0,19.7,18.8,-,1296,-,A,L,None,10.0,弱い
4,2019,9,17,火,盛岡,3300,にんじん,30300,岩 手,3.0,NaN,0.9,540,-,432,A,L,None,10.0,None


In [4]:
# やりたいこと
df[
    (df["品目名"] == "生しいたけ")
    & (df["産地名"] == "秋\u3000田")
    & (df["市場名"] == "大田")
]

,年,月,日,曜日,市場名,市場コード,品目名,品目コード,産地名,産地コード,品目計（ｔ：トン）,入荷量（ｔ：トン）,高値（円）,中値（円）,安値（円）,等級,階級,品名,重量（kg）,概況
662,2019,9,17,火,大田,13310,生しいたけ,38100,秋 田,5.0,NaN,3.5,194,-,173,A,M6,菌床,0.1,None
3745,2019,9,18,水,大田,13310,生しいたけ,38100,秋 田,5.0,NaN,1.4,173,140,140,A,8M,菌床,0.1,None
6718,2019,9,19,木,大田,13310,生しいたけ,38100,秋 田,5.0,NaN,1.5,173,140,140,A,8M,菌床,0.1,None
12717,2019,9,21,土,大田,13310,生しいたけ,38100,秋 田,5.0,NaN,1.8,173,140,140,A,8M,菌床,0.1,None
15764,2019,9,24,火,大田,13310,生しいたけ,38100,秋 田,5.0,NaN,3.7,194,-,173,A,M6,菌床,0.1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19284,2019,8,26,月,大田,13310,生しいたけ,38100,秋 田,5.0,NaN,2.1,194,162,151,A,M6,菌床,0.1,None
22397,2019,8,27,火,大田,13310,生しいたけ,38100,秋 田,5.0,NaN,1.6,173,140,108,A,8M,菌床,0.1,None
25496,2019,8,29,木,大田,13310,生しいたけ,38100,秋 田,5.0,NaN,1.9,194,173,162,A,M6,菌床,0.1,None
28607,2019,8,30,金,大田,13310,生しいたけ,38100,秋 田,5.0,NaN,1.5,173,140,140,A,8M,菌床,0.1,None


In [7]:
# 認証情報のファイルパス
credentials_path = "../../../.aws/credentials"

# ConfigParserオブジェクトを作成
config = configparser.ConfigParser()
config.read(credentials_path)

# プロファイル名を指定して認証情報を取得
profile_name = "default"  # 使用したいプロファイル名
aws_access_key_id = config[profile_name]["aws_access_key_id"]
aws_secret_access_key = config[profile_name]["aws_secret_access_key"]

In [13]:
# AWS設定
region = "ap-northeast-1"
database = "sampledb.mysample111"
query = """
SELECT *
FROM sampledb.mysample111
WHERE "市場名" = '大田'
  AND "品目名" = '生しいたけ'
  AND "産地名" = '秋　田';
"""
s3_output = "s3://mysample1111/Unsaved/"

# クライアントの作成
client = boto3.client(
    "athena",
    region_name=region,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

# クエリの実行
response = client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": database},
    ResultConfiguration={"OutputLocation": s3_output},
)

In [14]:
query_execution_id = response["QueryExecutionId"]

# クエリの実行状態をチェック
status = client.get_query_execution(QueryExecutionId=query_execution_id)[
    "QueryExecution"
]["Status"]["State"]

# 状態に応じて処理を行う
if status == "SUCCEEDED":
    result = client.get_query_results(QueryExecutionId=query_execution_id)
elif status in ["FAILED", "CANCELLED"]:
    print("Query failed or was cancelled with status:", status)
else:
    print("Query is still running with status:", status)

Query is still running with status: RUNNING


In [15]:
rows = result["ResultSet"]["Rows"]

# カラム名を取得
column_info = rows[0]["Data"]
columns = [col["VarCharValue"] for col in column_info]

# データを取得
data = []
for row in rows[1:]:
    data.append([col.get("VarCharValue", None) for col in row["Data"]])

# Pandasデータフレームに変換
df = pd.DataFrame(data, columns=columns)

# 不要なインデックス列が存在する場合は削除
if "__index_level_0__" in df.columns:
    df = df.drop(columns=["__index_level_0__"])

df

,年,月,日,曜日,市場名,市場コード,品目名,品目コード,産地名,産地コード,品目計（ｔ：トン）,入荷量（ｔ：トン）,高値（円）,中値（円）,安値（円）,等級,階級,品名,重量（kg）,概況
0,2020,6,6,土,大田,13310,生しいたけ,38100,秋 田,5.0,None,1.8,173,167,130,A,M6,菌床,0.1,None
1,2020,6,15,月,大田,13310,生しいたけ,38100,秋 田,5.0,None,2.2,173,140,130,A,M6,菌床,0.1,None
2,2020,1,5,日,大田,13310,生しいたけ,38100,秋 田,5.0,None,12.8,238,216,194,A,6M,菌床,0.1,None
3,2020,1,6,月,大田,13310,生しいたけ,38100,秋 田,5.0,None,3.1,216,162,162,A,6M,菌床,0.1,None
4,2020,1,9,木,大田,13310,生しいたけ,38100,秋 田,5.0,None,5.0,194,162,140,A,6M,菌床,0.1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2021,11,12,金,大田,13310,生しいたけ,38100,秋 田,5.0,None,3.3,173,148,130,A,M6,菌床,0.1,None
727,2021,11,13,土,大田,13310,生しいたけ,38100,秋 田,5.0,None,2.9,173,130,130,A,M6,菌床,0.1,None
728,2021,11,15,月,大田,13310,生しいたけ,38100,秋 田,5.0,None,4.7,173,148,130,A,M6,菌床,0.1,None
729,2020,6,1,月,大田,13310,生しいたけ,38100,秋 田,5.0,None,2.5,135,108,108,B,M6,菌床,0.1,None
